In [ ]:
!pip install gradio

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import os
import json
import gradio as gr

In [ ]:
class Classifier(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super(Classifier, self).__init__()
        self.model = models.densenet121(weights=None)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# Mount Google Drive
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Define the model
model = Classifier(num_classes=2)

# Load the saved model's state_dict (weights)
model_path = '/content/drive/MyDrive/final_model.pth'
checkpoint = torch.load(model_path)
class_mapping = checkpoint['class_mapping']
print(checkpoint.keys())

state_dict = checkpoint['model_state_dict']

# Create a new state_dict with modified keys
new_state_dict = {}
for key, value in state_dict.items():
    new_key = key.replace('model.', '')  # Remove 'model.' prefix
    new_state_dict[new_key] = value

# Load the new state dict into the model
model.model.load_state_dict(new_state_dict)
model.eval()  # Set the model to evaluation mode

# Preprocess the input image
image_path = '/content/drive/MyDrive/images.jpeg'  # Path to the input image
image = Image.open(image_path)

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Run the model on the image
with torch.no_grad():
    output = model(image_tensor)

_, predicted_class = torch.max(output, 1)

print(f"Predicted class index: {predicted_class.item()}")

# תוסיף:
predicted_label = class_mapping[predicted_class.item()]
print(f"Predicted label: {predicted_label}")


Mounted at /content/drive
dict_keys(['model_state_dict', 'optimizer_state_dict', 'num_classes', 'class_mapping'])
Predicted class index: 0
Predicted label: DR


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict(image):
    image = image.convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = F.softmax(output, dim=1)  # לחשב הסתברויות
        confidence, predicted_class = torch.max(probabilities, 1)
        predicted_label = class_mapping[predicted_class.item()]
    return f"{predicted_label} (Confidence: {confidence.item():.2%})"

# -------- הגדרת הממשק --------

iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="label",
    title="Image Classifier",
    description="Upload picture:"
)

iface.launch(share=False,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Using existing dataset file at: .gradio/flagged/dataset1.csv
Keyboard interruption in main thread... closing server.
